---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kpcrotty.github.io/)
    
Jones Graduate School of Business, Rice University

---


# U.S. STOCK MARKET RETURN

In [ ]:
from pandas_datareader import DataReader as pdr

mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[1]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt.index.name = "year"
mkt.name = "market"
mkt.index = mkt.index.astype(str)
mkt.tail()

# RETURNS FROM YAHOO FINANCE

In [ ]:
import yfinance as yf

ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]
price = price.resample("Y").last()
ser = price.pct_change()[:-1].dropna()
ser.index = ser.index.map(lambda x: x.year)
ser.index.name = "year"
ser.name = ticker
ser.tail()

# YAHOO RETURN OR MARKET

In [ ]:
flag = int(input("input 0 for market or 1 for yahoo return"))
data = ser if flag else mkt
name = f"{ticker.upper()} Return" if flag else "Market Return"

# FIGURE 1

In [ ]:
import plotly.graph_objects as go

trace = go.Scatter(
    x=data.index.to_list(),
    y=data, 
    mode="lines", 
    hovertemplate="%{x}<br>%{y:.1%}"
)
fig = go.Figure(trace)
fig.update_layout(
    xaxis_title="Year",
    yaxis_title=name,
    yaxis_tickformat=".0%",
    template="plotly_white",  
)
fig.show()

# FIGURE 2

In [ ]:
import numpy as np

mean = data.mean()
T = len(data)
geomean = (1+data).prod()**(1/T) - 1

trace1 = go.Scatter(
    x=data.index.to_list(),
    y=(1+data).cumprod(),
    mode="lines",
    name="Actual Accumulation",
    hovertemplate="%{y:.2f}"
)
trace2 = go.Scatter(
    x=data.index.to_list(),
    y=(1+mean)**np.arange(1, T+1),
    mode="lines",
    name="Compounded Arithmetic Avg",
    hovertemplate="%{y:.2f}"
)
trace3 = go.Scatter(
    x=data.index.to_list(),
    y=(1+geomean)**np.arange(1, T+1),
    mode="lines",
    name="Compounded Geometric Avg",
    hovertemplate="%{y:.2f}"
)

fig = go.Figure()
for trace in (trace1, trace2, trace3):
    fig.add_trace(trace)
fig.update_layout(
    xaxis_title="Year",
    hovermode="x unified",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
    ),
    template="plotly_white"
)
fig.show()